In [1]:
import sys
from enum import Enum, auto
sys.version

'3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]'

In [2]:
from enum import Enum, auto

class Color(Enum):
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

In [3]:
class Color(Enum):
    RED: int = auto()
    GREEN: int = auto()
    BLUE: int = auto()

In [4]:
c1 = Color(1)
c2 = Color(Color.RED)
c3 = Color(Color['RED'])
print(type(Color), type(c1), type(Color.RED), type(Color['RED']))

<class 'enum.EnumMeta'> <enum 'Color'> <enum 'Color'> <enum 'Color'>


In [5]:
Color(1) == 1             # false
Color.RED == Color(1)     # true
Color.RED == Color['RED'] # true

True

In [6]:
print(len(Color)) # 3
# can iterate:
for c in Color:
    print(c)

3
Color.RED
Color.GREEN
Color.BLUE


In [18]:
# Color(4) # ValueError: 4 is not a valid Color
# Color('RED') # ValueError: 'RED' is not a valid Color
# c1 = Color(1)
# c1 += 1 # TypeError: unsupported operand type(s) for +=: 'Color' and 'int'

In [8]:
class Color():     # not deriving from Enum
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

print(type(Color))

<class 'type'>


In [9]:
from enum import EnumMeta, Enum, auto

class Color(Enum):
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3
        
print(type(Color), type(Enum), type(EnumMeta))

<class 'enum.EnumMeta'> <class 'enum.EnumMeta'> <class 'type'>


In [10]:
class Enum(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        print(f'Defining new Enum type {cls}:')
        print(f'- metacls = {metacls}')
        print(f'- bases = {bases}')
        print(f'- classdict = {classdict}')

class Color(metaclass=Enum): # specifying our own Enum metaclass
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

Defining new Enum type Color:
- metacls = <class '__main__.Enum'>
- bases = ()
- classdict = {'__module__': '__main__', '__qualname__': 'Color', '__annotations__': {'RED': <class 'int'>, 'GREEN': <class 'int'>, 'BLUE': <class 'int'>}, 'RED': 1, 'GREEN': 2, 'BLUE': 3}


In [ ]:
class EnumMeta(type):
    ...

class Enum(metaclass=EnumMeta):
    ...

class Color(Enum):
    ...

In [13]:
class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

class Enum(metaclass=EnumMeta):
    pass

class Color(Enum):
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

print(Color._enumerations)                     # {'RED': 1, 'GREEN': 2, 'BLUE': 3}
print(type(Color), type(Enum), type(EnumMeta)) # ...
# Color(1)                                       # TypeError: Color() takes no arguments

{'RED': 1, 'GREEN': 2, 'BLUE': 3}
<class '__main__.EnumMeta'> <class '__main__.EnumMeta'> <class 'type'>


In [14]:
class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v

class Color(Enum):
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

Color(1) # okay
# Color(4) # ValueError: 4 is not a valid Color

In [15]:
class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
class Color(Enum):
    RED: int = 1
    GREEN: int = 2
    BLUE: int = 3

print(Color(1))
print(len(Color))
for c in Color:
    print(c)

Color.RED
3
Color.RED
Color.GREEN
Color.BLUE


In [16]:
class _Auto():
    pass

def auto():
    return _Auto()

class EnumMeta(type):
    def __new__(metacls, cls, bases, classdict, **kwds):
        enumerations = {x: y for x, y in classdict.items() if not x.startswith('__')}
        # handle auto()
        next_value = 1
        for k, v in enumerations.items():
            if type(v) != _Auto:
                next_value = v + 1
            else:
                enumerations[k] = next_value
                next_value += 1
        enum = super().__new__(metacls, cls, bases, classdict, **kwds)
        enum._enumerations = enumerations
        return enum

    def __len__(cls):
        return len(cls._enumerations)
    
    def __iter__(cls):
        return (cls(value) for value in cls._enumerations.values())
        
class Enum(metaclass=EnumMeta):
    def __init__(self, value):
        # make sure the passed in value is a valid enumeration value
        if value not in self.__class__._enumerations.values():
            raise ValueError(f'{value} is not a valid {self.__class__.__name__}')
        # save the actual enumeration value
        for k, v in self.__class__._enumerations.items():
            if v == value:
                self.__key = k
                self.__value = v
    
    def __str__(self):
        return "%s.%s" % (self.__class__.__name__, self.__key)
    
class Color(Enum):
    RED: int = auto()
    GREEN: int = auto()
    BLUE: int = auto()

print(Color(1))
print(len(Color))
for c in Color:
    print(c)

Color.RED
3
Color.RED
Color.GREEN
Color.BLUE
